# Uploading CAN data to InfluxDB

In [ ]:
import csv
from datetime import datetime, timezone
import time
import cantools
from influxdb_client import Point, InfluxDBClient
from influxdb_client.client.write_api import WriteOptions

def parse_can_csv_row(row, db):
    """
    Parse one CSV row dict, decode it via cantools, and return a list of InfluxDB Points.
    """
    try:
        can_id_int = int(row['message_id'])
    except (KeyError, ValueError):
        return [f"Error: Invalid CAN ID '{row.get('message_id')}'"]

    try:
        message = db.get_message_by_frame_id(can_id_int)
    except KeyError:
        return [f"Error: No message found for CAN ID {can_id_int}"]

    # collect the eight bytes (some may be empty strings)
    data_list = bytes(
        int(row[f'byte{b}']) for b in range(8)
        if row.get(f'byte{b}') not in (None, '')
    )

    try:
        decoded = message.decode(data_list)
    except Exception as e:
        return [f"Error: Decoding error for CAN ID {can_id_int} — {e}"]

    # parse the timestamp field into a UTC datetime
    try:
        ts = float(row['timestamp'])
        ts_dt = datetime.fromtimestamp(ts, tz=timezone.utc)
    except Exception:
        ts_dt = datetime.now(timezone.utc)

    points = []
    for sig_name, raw in decoded.items():
        sig = message.get_signal_by_name(sig_name)
        desc = getattr(sig, 'comment', '') or "No description"
        unit = getattr(sig, 'unit', '') or "N/A"
        if hasattr(raw, 'value'):  # NamedSignalValue
            val = float(raw.value)
            label = raw.name
        else:
            val = float(raw)
            label = str(raw)

        pt = (
            Point("canBus")
            .tag("signalName", sig_name)
            .tag("messageName", message.name)
            .tag("canId", str(can_id_int))
            .field("sensorReading", val)
            .field("unit", unit)
            .field("description", desc)
            .field("signalLabel", label)
            .time(ts_dt)
        )
        points.append(pt)
    return points

def process_csv(file_path, db, write_api, bucket, mps=400):
    """
    Read CSV, decode at `mps` messages/sec, batch‑write to InfluxDB.
    """
    batch = []
    count = 0
    start = time.time()

    with open(file_path, newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        for idx, row in enumerate(reader):
            pts = parse_can_csv_row(row, db)
            if pts and isinstance(pts[0], str) and pts[0].startswith("Error:"):
                print(f"Row {idx}: {pts[0]}")
            else:
                batch.extend(pts)
                count += 1

            if count >= mps:
                if batch:
                    write_api.write(bucket=bucket, org="WFR", record=batch)
                    batch = []
                elapsed = time.time() - start
                if elapsed < 1:
                    time.sleep(1 - elapsed)
                start = time.time()
                count = 0

    if batch:
        write_api.write(bucket=bucket, org="WFR", record=batch)

def main():
    # load DBC
    dbc_file = 'WFR25-2.dbc'
    try:
        db = cantools.database.load_file(dbc_file)
        print(f"Loaded DBC: {dbc_file}")
    except Exception as e:
        print(f"Failed loading DBC: {e}")
        return

    # InfluxDB client setup
    influx_url = "http://3.98.181.12:8086"
    with open('influx_token.txt') as f:
        token = f.read().strip()

    client = InfluxDBClient(url=influx_url, token=token, org="WFR")
    write_api = client.write_api(write_options=WriteOptions(batch_size=10000,
                                                            flush_interval=1000))

    csv_path = 'local_analysis/cleaned_can.csv'
    process_csv(csv_path, db, write_api, bucket="ourCar", mps=10000)
    print("Finished writing all points.")

if __name__ == "__main__":
    main()